# 比赛之前的准备
- 引入所需模块
- 设置一些参数

In [1]:
import matplotlib.pyplot as plt
import random, time, sys, os

plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False

Times = 100000
Map_Size = 24
Win_Rate = {"卡卡罗": 0, "今汐": 0, "长离": 0, "守岸人": 0, "椿": 0, "珂莱塔": 0, "洛可可": 0, "菲比": 0, "布兰特": 0, "坎特蕾拉": 0, "赞妮": 0, "卡提希娅": 0}
Cubes = {"A": ["卡卡罗", "今汐", "长离", "守岸人", "椿", "珂莱塔"], "B": ["洛可可", "菲比", "布兰特", "坎特蕾拉", "赞妮", "卡提希娅"]}

# 地图操作
- 地图清空
- 地图初始化

In [2]:
def Map_Reset() -> list[list]:
    return [[] for i in range(Map_Size + 3)]

def Map_Init(cubes: list[str], mode: int) -> list[list]:
    map = Map_Reset()
    if mode == 0: # 上半场比赛
        map[3] = [cube for cube in cubes]
        random.shuffle(map[3])
    elif mode == 1: # 下半场比赛
        map[3], map[3 - len(cubes) // 2] = [cubes[0]], [cubes[-1]]
        for i in range(2, 3 - len(cubes) // 2, -1): # 初始化中间部分
            map[i] = cubes[(6 - i * 2): (4 - i * 2): -1]
    return map

# 地图查询
- 获取指定团子位置
- 获取指定团子排名
- 获取当前排行榜

In [3]:
def Get_Map_Pos(cube: str, map: list[list]) -> int:
    for i in range(len(map)): # 从前往后搜
        if cube in map[i]: # 在此堆中
            return i

def Get_Rank(cube: str, map: list[list]) -> int:
    rank = 1
    for i in range(len(map) - 1, -1, -1): # 从后往前搜
        for j in range(len(map[i]) - 1, -1, -1): # 从上往下看
            if cube == map[i][j]:
                return rank
            rank += 1

def Get_Ranks(map: list[list]) -> list[str]:
    ranks = []
    for i in range(len(map) - 1, -1, -1): # 从后往前搜
        ranks += map[i][::-1]
    return ranks

# 步骤化的比赛
- 决定行动顺序
- 决定步数
- 指定团子移动

In [4]:
def Decide_Turn(cubes: list[str], map: list[list], skill: dict, mode: int) -> tuple[list[str], dict]:
    if mode == 0: # 上半场首轮
        cube_turn = map[3][::-1]
    elif mode == 1: # 其他轮次
        cube_turn = [cube for cube in cubes]
        random.shuffle(cube_turn)
    if "长离" in cubes and skill["长离"]: # 长离在时才能发动技能
        cube_turn.remove("长离")
        cube_turn.append("长离")
        skill["长离"] = False
    return (cube_turn, skill)

def Decide_Step(cube: str, map: list[list], skill: dict, cube_turn: list[str]) -> tuple[int, dict]:
    if cube == "守岸人": step = random.randint(2, 3) # 守岸人
    elif cube == "赞妮": step = random.randint(1, 2) * 2 - 1 # 赞妮
    else: step = random.randint(1, 3) # 其他人
    if cube == "卡卡罗": # 卡卡罗
        if Get_Rank(cube = cube, map = map) == len(cube_turn): # 技能触发
            step += 3
    elif cube == "椿": # 椿
        if random.randint(1, 2) == 1 and len(map[Get_Map_Pos(cube = cube, map = map)]) > 1 and not skill[cube]: # 技能触发
            step += len(map[Get_Map_Pos(cube = cube, map = map)]) - 1
            skill[cube] = True
    elif cube == "珂莱塔": # 珂莱塔
        if random.randint(1, 25) <= 7: # 技能触发
            step *= 2
    elif cube == "洛可可": # 洛可可
        if cube_turn[-1] == cube: # 技能触发
            step += 2
    elif cube == "菲比": # 菲比
        if random.randint(1, 2) == 1: # 技能触发
            step += 1
    elif cube == "布兰特": # 布兰特
        if cube_turn[0] == cube: # 技能触发
            step += 2
    elif cube == "赞妮": # 赞妮
        if skill[cube]: # 技能触发
            step += 2
            skill[cube] = False
        if len(map[Get_Map_Pos(cube = cube, map = map)]) > 1 and random.randint(1, 5) <= 2 and not skill[cube]: # 技能判定
            skill[cube] = True
    elif cube == "卡提希娅": # 卡提希娅
        if skill[cube] and random.randint(1, 5) <= 3: # 技能触发
            step += 2
    return (step, skill)

def Skill(cube: str, map: list[list], skill: dict, cube_turn: list[str], mode: int) -> tuple[list[list], dict]:
    if "今汐" in cube_turn: # 今汐在场时才能触发
        if (mode == 1 and cube != "今汐") or (mode == 0 and cube_turn.index(cube) > cube_turn.index("今汐")): # 首轮行动后才触发
            map_pos_jx = Get_Map_Pos(cube = "今汐", map = map)
            if map[map_pos_jx].index("今汐") < len(map[map_pos_jx]) - 1 and random.randint(1, 5) <= 2: # 头上有其他团子时40%概率触发
                map[map_pos_jx].remove("今汐")
                map[map_pos_jx].append("今汐")
    if "长离" in cube_turn: # 长离在场时才能触发
        if map[Get_Map_Pos(cube = "长离", map = map)].index("长离") > 0 and random.randint(1, 20) <= 13 and not skill[cube]: # 底下有其他团子时65%概率触发
            skill["长离"] = True
    if "卡提希娅" in cube_turn: # 卡提希娅在场时才能触发
        if cube == "卡提希娅" and Get_Rank(cube = cube, map = map) == len(cube_turn) and not skill[cube]: # 技能触发
            skill[cube] = True
    return (map, skill)

def Move(cube: str, map: list[list], skill: dict, step: int) -> tuple[list[list], dict]:
    # 决定起点终点
    src = Get_Map_Pos(cube = cube, map = map)
    dst = src + step
    if dst >= len(map): # 超出地图
        dst = len(map) - 1
    # 移动
    stack = []
    if cube == "椿" and skill[cube]: # 椿技能触发
        map[src].remove(cube)
        map[dst].append(cube)
        skill[cube] = False
    elif cube == "坎特蕾拉" and not skill[cube]: # 坎特蕾拉技能未触发
        while map[src][-1] != cube: # 上方的团子一同移动
            stack.append(map[src].pop())
        stack.append(map[src].pop())
        for i in range(src + 1, dst + 1): # 检查路径
            if map[i] != []: # 技能触发
                skill[cube] = True
                break
        stack += map[i][::-1]
        map[i] = []
        map[dst] += stack[::-1]
    else: # 其他团子
        while map[src][-1] != cube: # 上方的团子一同移动
            stack.append(map[src].pop())
        stack.append(map[src].pop())
        map[dst] += stack[::-1]
    return (map, skill)

# 整体化的比赛
- 单轮行动
- 半场比赛
- 全场比赛
- 小团快跑

In [5]:
def Game_Cycle(cubes: list[str], map: list[list], skill: dict, mode: int) -> tuple[list[list], dict, bool]:
    # 决定行动顺序
    cube_turn, skill = Decide_Turn(cubes = cubes, map = map, skill = skill, mode = mode)
    for cube in cube_turn: # 依次行动
        # 决定步数、移动
        step, skill = Decide_Step(cube = cube, map = map, skill = skill, cube_turn = cube_turn)
        map, skill = Move(cube = cube, map = map, skill = skill, step = step)
        # 胜利判定、技能发动
        if map[-1] != []:
            return (map, skill, True)
        map, skill = Skill(cube = cube, map = map, skill = skill, cube_turn = cube_turn, mode = mode)
    return (map, skill, False)

def Game_Half(cubes: list[str], map: list[list], mode: int) -> list[str]:
    # 重置技能
    skill = {}
    for cube in cubes: # 依次初始化
        skill[cube] = False
    # 轮次比赛
    map, skill, res = Game_Cycle(cubes = cubes, map = map, skill = skill, mode = mode)
    while not res: # 比赛尚未结束
        map, skill, res = Game_Cycle(cubes = cubes, map = map, skill = skill, mode = 1)
    return Get_Ranks(map = map)

def Game_Whole(cubes: list[str]) -> list[str]:
    # 上半场
    map = Map_Init(cubes = cubes, mode = 0)
    ranks = Game_Half(cubes = cubes, map = map, mode = 0)
    # 下半场
    map = Map_Init(cubes = ranks, mode = 1)
    ranks = Game_Half(cubes = cubes, map = map, mode = 1)
    return ranks

def Running_Cubes(cubes: dict) -> str:
    # A组B组小组赛
    cubes_a = Game_Whole(cubes = cubes["A"])[0: -2]
    cubes_b = Game_Whole(cubes = cubes["B"])[0: -2]
    # A组B组淘汰赛
    cubes_a = Game_Whole(cubes = cubes_a)[0: -2]
    cubes_b = Game_Whole(cubes = cubes_b)[0: -2]
    # 总决赛
    cubes_final = Game_Whole(cubes = cubes_a + cubes_b)[0]
    return cubes_final

# 比赛之后的展示
- 绘制胜率饼状图
- 生成进度条

In [6]:
def Show_Win_Rate(win_rate: dict) -> None:
    # 获取团子列表、胜率列表
    win_rate = sorted(win_rate.items(), key = lambda k: k[1], reverse = True)
    cubes = [win_rate[i][0] for i in range(len(win_rate))]
    rates = [win_rate[i][1] for i in range(len(win_rate))]
    # 绘制饼状图、展示
    plt.pie(x = rates, labels = cubes, autopct = "%1.1f%%")
    plt.title("小团快跑冠军获得率")
    plt.show()

def Status_Bar(status: int) -> str:
    status = status * 100 // Times + 1
    if status <= 9: status_bar = "[" + "=" * (status // 2) + " " * (25 - status // 2) + str(status) + "%" + " " * 23 + "]" # 一位数
    elif status <= 47: status_bar = "[" + "=" * (status // 2) + " " * (24 - status // 2) + str(status) + "%" + " " * 23 + "]" # 两位数·一
    elif status <= 55: status_bar = "[" + "=" * 24 + str(status) + "%" + " " * 23 + "]" # 两位数·二
    elif status <= 99: status_bar = "[" + "=" * 24 + str(status) + "%" + "=" * (status // 2 - 27) + " " * (50 - status // 2) + "]" # 两位数·三
    elif status == 100: status_bar = "[" + "=" * 23 + "100%" + "=" * 23 + "]" # 三位数
    return status_bar

# 主函数、运行

In [7]:
def main() -> None: # 主函数
    time_src = time.time()
    for i in range(Times): # 十万次模拟
        winner = Running_Cubes(cubes = Cubes)
        Win_Rate[winner] += 1
        # 刷新进度条
        time_dst = time.time()
        if time_dst - time_src >= 1: # 刷新进度条
            time_src = time_dst
            status_bar = Status_Bar(status = i)
            os.system("clear")
            sys.stdout.write("\r模拟进度: ")
            sys.stdout.write(status_bar)
    Show_Win_Rate(win_rate = Win_Rate)

if __name__ == "__main__": # 运行主函数
    main()

模拟进度: [======                  12%                       ]

: 